<a href="https://colab.research.google.com/github/ADionysopoulos/efood_assesment/blob/main/BigQuery_bquxjob_4268ce04_18b9ad7f5ea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import numpy as np
import pandas as pd
from itertools import product

project = 'efood2023-404109' # Project ID inserted based on the query results selected to explore
location = 'EU' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

MessageError: ignored

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=efood2023-404109:EU:bquxjob_4268ce04_18b9ad7f5ea)
back to BigQuery to edit the query within the BigQuery user interface.

In [ ]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
print(job.query)

# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()

In [ ]:
results_init["device"].unique()

In [ ]:
# Keep only the users who had took coupon in the past
results = results_init
results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
results = results.set_index('user_id')
target_coupon_results = results.loc[users_with_coupon,:]
target_coupon_results = target_coupon_results.reset_index()

In [ ]:
coupon_effect = pd.pivot_table(target_coupon_results,
                               index = ["user_id", "order_timestamp_day"],
                               values = ["order_id", "amount", "coupon_discount_amount"],
                               aggfunc= {"order_id": "count", "amount": [np.median, np.sum], "coupon_discount_amount": np.sum})
coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

In [ ]:
user_ids = coupon_effect.reset_index()['user_id'].unique()
days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
df_index_day     = [day for _ in user_ids for day in days_range]

In [ ]:
extended_coupon_effect = pd.DataFrame(columns = coupon_effect.columns)
extended_coupon_effect["user_id"] = df_index_user_id
extended_coupon_effect["order_timestamp_day"] = df_index_day
extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

In [ ]:
extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
extended_coupon_effect = extended_coupon_effect.fillna(0)
extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

In [ ]:
A = extended_coupon_effect.reset_index()
events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

coupon_case = 0
coupon_window = 3

extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

for index, row in events.iterrows():

  user_id = row["user_id"]
  coupon_day = row["order_timestamp_day"]

  if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

    index_prev= []

    for k in range(coupon_day - coupon_window, coupon_day):
      index_prev.append([user_id, k])

    index_after= []

    for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
      index_after.append([user_id, k])

    if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
      extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
      extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

    coupon_case +=1



In [ ]:
A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})

In [ ]:
A1.columns
diffs = A2 - A1
diffs.describe()

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()

for i in results_init['user_class_name'].unique():

  results_init = job.to_dataframe()
  results_init = results_init[results_init['user_class_name'] == i ]

  results = results_init
  # Keep only the users who had took coupon in the past
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
  users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
  results = results.set_index('user_id')
  target_coupon_results = results.loc[users_with_coupon,:]
  target_coupon_results = target_coupon_results.reset_index()

  coupon_effect = pd.pivot_table(target_coupon_results,
                                index = ["user_id", "order_timestamp_day"],
                                values = ["order_id", "amount", "coupon_discount_amount"],
                                aggfunc= {"order_id": "count", "amount": [np.median, np.sum],"coupon_discount_amount": np.sum})
  coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

  user_ids = coupon_effect.reset_index()['user_id'].unique()
  days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

  df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
  df_index_day = [day for _ in user_ids for day in days_range]

  extended_coupon_effect =pd.DataFrame(columns = coupon_effect.columns)
  extended_coupon_effect["user_id"] = df_index_user_id
  extended_coupon_effect["order_timestamp_day"] = df_index_day
  extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

  extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
  extended_coupon_effect = extended_coupon_effect.fillna(0)
  extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

  A = extended_coupon_effect.reset_index()
  events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

  coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
  coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

  coupon_case = 0
  coupon_window = 3

  extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
  extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

  for index, row in events.iterrows():

    user_id = row["user_id"]
    coupon_day = row["order_timestamp_day"]

    if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

      index_prev= []

      for k in range(coupon_day - coupon_window, coupon_day):
        index_prev.append([user_id, k])

      index_after= []

      for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
        index_after.append([user_id, k])

      if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
        extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
        extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

      coupon_case +=1

  A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
  A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})


  diffs = A2 - A1
  print(i)

  print(diffs.describe())
  print("----------------------------------------------------------------")


In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()
mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
results_init = results_init[mask]

for i in results_init['city'].unique():

  results_init = job.to_dataframe()
  mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
  results_init = results_init[mask]
  results_init = results_init[results_init['city'] == i ]

  results = results_init
  # Keep only the users who had took coupon in the past
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
  users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
  results = results.set_index('user_id')
  target_coupon_results = results.loc[users_with_coupon,:]
  target_coupon_results = target_coupon_results.reset_index()

  coupon_effect = pd.pivot_table(target_coupon_results,
                                index = ["user_id", "order_timestamp_day"],
                                values = ["order_id", "amount", "coupon_discount_amount"],
                                aggfunc= {"order_id": "count", "amount": [np.median, np.sum],"coupon_discount_amount": np.sum})
  coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

  user_ids = coupon_effect.reset_index()['user_id'].unique()
  days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

  df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
  df_index_day = [day for _ in user_ids for day in days_range]

  extended_coupon_effect =pd.DataFrame(columns = coupon_effect.columns)
  extended_coupon_effect["user_id"] = df_index_user_id
  extended_coupon_effect["order_timestamp_day"] = df_index_day
  extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

  extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
  extended_coupon_effect = extended_coupon_effect.fillna(0)
  extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

  A = extended_coupon_effect.reset_index()
  events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

  coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
  coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

  coupon_case = 0
  coupon_window = 3

  extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
  extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

  for index, row in events.iterrows():

    user_id = row["user_id"]
    coupon_day = row["order_timestamp_day"]

    if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

      index_prev= []

      for k in range(coupon_day - coupon_window, coupon_day):
        index_prev.append([user_id, k])

      index_after= []

      for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
        index_after.append([user_id, k])

      if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
        extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
        extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

      coupon_case +=1

  A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
  A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})

  A1.columns
  diffs = A2 - A1
  print(i)
  print(diffs.describe())
  print("----------------------------------------------------------------")

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()
mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
results_init = results_init[mask]
mask2 = (results_init["device"] != "Android")
results_init.loc[mask2, "device"] = "Not Android"

for i in results_init['device'].unique():

  results_init = job.to_dataframe()
  mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
  results_init = results_init[mask]
  mask2 = (results_init["device"] != "Android")
  results_init.loc[mask2, "device"] = "Not Android"
  results_init = results_init[results_init['device'] == i ]

  results = results_init
  # Keep only the users who had took coupon in the past
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
  users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
  results = results.set_index('user_id')
  target_coupon_results = results.loc[users_with_coupon,:]
  target_coupon_results = target_coupon_results.reset_index()

  coupon_effect = pd.pivot_table(target_coupon_results,
                                index = ["user_id", "order_timestamp_day"],
                                values = ["order_id", "amount", "coupon_discount_amount"],
                                aggfunc= {"order_id": "count", "amount": [np.median, np.sum],"coupon_discount_amount": np.sum})
  coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

  user_ids = coupon_effect.reset_index()['user_id'].unique()
  days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

  df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
  df_index_day = [day for _ in user_ids for day in days_range]

  extended_coupon_effect =pd.DataFrame(columns = coupon_effect.columns)
  extended_coupon_effect["user_id"] = df_index_user_id
  extended_coupon_effect["order_timestamp_day"] = df_index_day
  extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

  extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
  extended_coupon_effect = extended_coupon_effect.fillna(0)
  extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

  A = extended_coupon_effect.reset_index()
  events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

  coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
  coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

  coupon_case = 0
  coupon_window = 3

  extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
  extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

  for index, row in events.iterrows():

    user_id = row["user_id"]
    coupon_day = row["order_timestamp_day"]

    if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

      index_prev= []

      for k in range(coupon_day - coupon_window, coupon_day):
        index_prev.append([user_id, k])

      index_after= []

      for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
        index_after.append([user_id, k])

      if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
        extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
        extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

      coupon_case +=1

  A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
  A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})

  A1.columns
  diffs = A2 - A1
  print(i)
  print(diffs.describe())
  print("----------------------------------------------------------------")

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()
mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
results_init = results_init[mask]

for i in results_init['paid_cash'].unique():

  results_init = job.to_dataframe()
  mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
  results_init = results_init[mask]
  results_init = results_init[results_init['paid_cash'] == i]

  results = results_init
  # Keep only the users who had took coupon in the past
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
  users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
  results = results.set_index('user_id')
  target_coupon_results = results.loc[users_with_coupon,:]
  target_coupon_results = target_coupon_results.reset_index()

  coupon_effect = pd.pivot_table(target_coupon_results,
                                index = ["user_id", "order_timestamp_day"],
                                values = ["order_id", "amount", "coupon_discount_amount"],
                                aggfunc= {"order_id": "count", "amount": [np.median, np.sum],"coupon_discount_amount": np.sum})
  coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

  user_ids = coupon_effect.reset_index()['user_id'].unique()
  days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

  df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
  df_index_day = [day for _ in user_ids for day in days_range]

  extended_coupon_effect =pd.DataFrame(columns = coupon_effect.columns)
  extended_coupon_effect["user_id"] = df_index_user_id
  extended_coupon_effect["order_timestamp_day"] = df_index_day
  extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

  extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
  extended_coupon_effect = extended_coupon_effect.fillna(0)
  extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

  A = extended_coupon_effect.reset_index()
  events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

  coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
  coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

  coupon_case = 0
  coupon_window = 3

  extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
  extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

  for index, row in events.iterrows():

    user_id = row["user_id"]
    coupon_day = row["order_timestamp_day"]

    if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

      index_prev= []

      for k in range(coupon_day - coupon_window, coupon_day):
        index_prev.append([user_id, k])

      index_after= []

      for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
        index_after.append([user_id, k])

      if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
        extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
        extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

      coupon_case +=1

  A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
  A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})

  A1.columns
  diffs = A2 - A1
  print(i)
  print(diffs.describe())
  print("----------------------------------------------------------------")